In [18]:
# pip install cmake
# pip install dlib
# pip install face-recognition
# pip install opencv-python

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


## Import all the necessary libraries


In [1]:
import cv2
import numpy as np
import face_recognition
import os
import pickle
from datetime import datetime
import csv

In [2]:
path = 'students'
images = []
classNames = []
ImgDirs = os.listdir(path)
for imgDir in ImgDirs:
    curImg = cv2.imread(f'{path}/{imgDir}')
    images.append(curImg)
    classNames.append(os.path.splitext(imgDir)[0])

# print('images', images)
print('class', classNames)

class ['Ayesha', 'bijoy', 'Dorothi', 'Popeye', 'Snigdha']


## Save the class names

In [3]:
def save_class_names(known_encodings, save_file):
    with open(save_file, "wb") as file:
        pickle.dump(known_encodings, file)
save_file = "classNames.pkl"
save_class_names(classNames,save_file)

## Load the class names

In [4]:
save_file = "classNames.pkl"
def load_known_encodings(load_file):
    with open(load_file, "rb") as file:
        data = pickle.load(file)
    return data
classNames= load_known_encodings(save_file)
print(classNames)

['Ayesha', 'bijoy', 'Dorothi', 'Popeye', 'Snigdha']


## Encode the image that are provided

In [5]:
def findEncoding(images):
    encodeList = []

    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

encodeListKnown = findEncoding(images)
print(len(encodeListKnown))

5


## Save the encoding model

In [6]:
def save_known_encodings(known_encodings, save_file):
    with open(save_file, "wb") as file:
        pickle.dump(known_encodings, file)

save_file = "encodeListKnown.pkl"
save_known_encodings(encodeListKnown, save_file)

## Load the encoding model

In [7]:
save_file = "encodeListKnown.pkl"
def load_known_encodings(load_file):
    with open(load_file, "rb") as file:
        data = pickle.load(file)
    return data
encodeListKnown = load_known_encodings(save_file)

## Attendance Function that are saved in csv file

In [8]:
def markAttendance(name,date):
    file_name = f'Attendance_Folder/Attendance_{date}.csv'
    if not os.path.exists(file_name):
        with open(file_name, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Name', 'Timestamp'])

    with open(file_name, 'r+') as f:
        myDataList = f.readlines()
        nameList = []

        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')


## RealTime Attendance 

In [9]:
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()

    imgS = cv2.resize(img,(0,0), None, 1, 1)
    imgS = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    faceCurFrame = face_recognition.face_locations(imgS)
    encodeCurFrame = face_recognition.face_encodings(imgS, faceCurFrame)

    for encodeFace, faceLoc in zip(encodeCurFrame, faceCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        # print(faceDis)

        matchIndex = np.argmin(faceDis)
            
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            print(name)
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 , x2 , y2 , x1 

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            current_date = datetime.now().strftime('%Y-%m-%d')
            markAttendance(name, current_date)

    cv2.imshow('Attendance', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

POPEYE
POPEYE
POPEYE
POPEYE
POPEYE
POPEYE
POPEYE
POPEYE
POPEYE


## Building a simple app of Attendance

In [3]:
import tkinter as tk
import cv2
import face_recognition
import pickle
from datetime import datetime
import csv
from PIL import Image, ImageTk
import os
import numpy as np

def load_known_encodings(load_file):
    with open(load_file, "rb") as file:
        data = pickle.load(file)
    return data

def markAttendance(name, date):
    file_name = f'Attendance_Folder/Attendance_{date}.csv'
    if not os.path.exists(file_name):
        with open(file_name, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Name', 'Timestamp'])

    with open(file_name, 'r+') as f:
        myDataList = f.readlines()
        nameList = []

        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')

def recognize_faces():
    success, img = cap.read()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    faceCurFrame = face_recognition.face_locations(imgS)
    encodeCurFrame = face_recognition.face_encodings(imgS, faceCurFrame)

    for encodeFace, faceLoc in zip(encodeCurFrame, faceCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)

        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            current_date = datetime.now().strftime('%Y-%m-%d')
            markAttendance(name, current_date)

    imgtk = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imgtk = ImageTk.PhotoImage(image=Image.fromarray(imgtk))
    label_video.imgtk = imgtk
    label_video.configure(image=imgtk)
    label_video.after(10, recognize_faces)

def start_attendance():
    global cap,img
    cap = cv2.VideoCapture(0)
    recognize_faces()

def stop_attendance():
    global cap
    if cap is not None:
        cap.release()
    cv2.destroyAllWindows()

classNames_file = "classNames.pkl"
encodeListKnown_file = "encodeListKnown.pkl"

classNames = load_known_encodings(classNames_file)
encodeListKnown = load_known_encodings(encodeListKnown_file)

app = tk.Tk()
app.title("Face Recognition Attendance App")

label_video = tk.Label(app)
label_video.pack()

button_start = tk.Button(app, text="Start Attendance", command=start_attendance)
button_start.pack()

button_stop = tk.Button(app, text="Stop Attendance", command=stop_attendance)
button_stop.pack()

app.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\excal\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\excal\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 839, in callit
    func(*args)
  File "C:\Users\excal\AppData\Local\Temp\ipykernel_15956\3688076297.py", line 37, in recognize_faces
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
cv2.error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

